# Women's Health Study Accelerometry Data

This notebook goes over the code needed to reproduce the results from Table 3, using WiSER with the Women's Health Study (WHS) accelerometry data. We also compare its use to fitting a linear mixed effects model via MixedModels.jl.

#### Packages and Reproducibility

Julia allows for easy reproducibility, by including a `Manifest.toml` and `Project.toml` pair, the user can simply run `] activate .` and the correct environment with dependencies used will run.  

In [1]:
]activate .

 Activating environment at `~/WiSER_Reproduce/womens_health_study_accelerometry_analysis/Project.toml`


Note: We use the KNITRO solver in our analysis, which requires a KNITRO license. If you wish to run the analysis without it, you can use another solver, but the results will be slightly different. Commented code is given to do this.

## Availability & Description

Due to confidentiality concerns, access to the WHS Accelerometry dataset is only available through the National Institutes of Health (NIH) database of Genotypes and Phenotypes (dbGaP). Researchers can apply for acceess to download this dataset through dbGaP. 

The URL for the webpage is https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=phs001964.v1.p1 and the dbGaP Study Accession identifier is phs001964.v1.p1. This page includes a description of the dataset, study, and details on how to request access to the data. We cannot give more details on the data due to dbGaP's data use agreement. 

Due to data confidentiality concerns, we supress output of the dataframes that show subject-level data. 

This notebook goes over code, that when used with the dbGAP's WHS Accelerometry data, can reproduce results in the paper (Table 3).

In [2]:
versioninfo()

Julia Version 1.4.0
Commit b8e9a9ecc6 (2020-03-21 16:36 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i9-9920X CPU @ 3.50GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)
Environment:
  JULIA_NUM_THREADS = 8


## Data Cleaning

The following goes through the steps to clean the data once it is downloaded from dbGaP.

In [3]:
using DataFrames, CSV, StatsBase, Statistics, CodecZlib, Dates
ENV["COLUMNS"]=1000

1000

Import the data and add some variables. 

The following assumes you have downloaded the data from dbGaP and are in the downloaded data folder:

In [4]:
pathofdata = "WHS_Accelerometer_phs001964/PhenoGenotypeFiles/RootStudyConsentSet_phs001964.WHS_Accelerometer.v1.p1.c1.GRU/PhenotypeFiles/"

accelerometer_subject = open(pathofdata * "phs001964.v1.pht009959.v1.p1.WHS_Accelerometer_Subject.MULTI.txt.gz") do io
    DataFrame!(CSV.File(GzipDecompressorStream(io), delim="\t", comment = "#", ignoreemptylines=true))
    end

accelerometer_mins = open(pathofdata * "phs001964.v1.pht009963.v1.p1.c1.WHS_Accelerometer_60sec.GRU.txt.gz") do io
#     CSV.read(GzipDecompressorStream(io), delim="\t", comment = "#", ignoreemptylines=true)
    DataFrame!(CSV.File(GzipDecompressorStream(io), delim="\t", comment = "#", ignoreemptylines=true))
end

sum_data = open(pathofdata * "phs001964.v1.pht009960.v1.p1.c1.WHS_Accelerometer_d20180514_Pub.GRU.txt.gz") do io
#      CSV.read(GzipDecompressorStream(io), delim="\t", comment = "#", ignoreemptylines=true)
    DataFrame!(CSV.File(GzipDecompressorStream(io), delim="\t", comment = "#", ignoreemptylines=true))
end


smoking_pub = open(pathofdata * "phs001964.v1.pht009961.v1.p1.c1.WHS_Accelerometer_Smoking_Pub.GRU.txt.gz") do io
     DataFrame!(CSV.File(GzipDecompressorStream(io), delim="\t", comment = "#", ignoreemptylines=true))
end

# get hour of day 
accelerometer_mins[!, :hour] = Dates.hour.(accelerometer_mins[!, :timeHMS]) ;
# get minute of the day
accelerometer_mins[!, :mins] = Dates.minute.(accelerometer_mins[!, :timeHMS]);
# group every 5 minutes for each hour
accelerometer_mins[!, :mingroup] = floor.(accelerometer_mins[!, :mins] / 5) #every 5 minutes
# create smoking variable
smoking_pub[!, :smoker] = map(x -> ismissing(x) ? missing : x == 1 ? "never" : x == 2 ? "past" : "current",
    smoking_pub[!, :smoke])
# create race variable from numeric definitions 
sum_data[!, :RACE] = map(x -> ismissing(x) ? missing : x == 1 ? "white" : x == 2 ? "hispanic" : 
    x == 3 ? "african american" : x == 4 ? "asian" : x == 5 ? "native american" : "other",
    sum_data[!, :RACE]);


# subset to just these variables

keepvars = [:dbGaP_Subject_ID
 :Subject_ID
 :wday
 :ep60_maxsteps
 :m_total
 :sum_valid
:EP60_total_steps
 :compliant
:RACE
:stairs
 :genhealth
 :bmi
 :ageaccel
 :day_worn
 :season]

# get data we need from the daily summary data
sum_data = sum_data[!, keepvars];

┌ Warning: `CSV.File` or `CSV.Rows` with `TranscodingStreams.TranscodingStream{GzipDecompressor,IOStream}` object is deprecated; pass a filename, `IOBuffer`, or byte buffer directly (via `read(x)`)
│   caller = Header at header.jl:123 [inlined]
└ @ Core /home/cgerman/.julia/packages/CSV/MKemC/src/header.jl:123


In [5]:
# summarize data by hour, to get steps and vector magnitude 
summarized_hour = combine(DataFrames.groupby(accelerometer_mins, 
        [:Subject_ID, :wday, :hour]), :steps => sum, :count_vm => sum);

In [6]:
#Get best top p hours 
function findtop_p(x, p)
    sortedx = sort(x, rev=true)
    return ind1 = findfirst(x .== sortedx[p]) - 1 #hours start at 1, subtract 1 to get true hour 
end

# get max hour for each person, each day 
maxhours = combine(DataFrames.groupby(summarized_hour, 
        [:Subject_ID, :wday]),
        :count_vm_sum => (x -> findtop_p(x, 1)) => :maxvm_hour1,
        :count_vm_sum => (x -> findtop_p(x, 2)) => :maxvm_hour2);

In [7]:
# get the minute data for the top hour found above for each person, each day
tophour_data = DataFrames.innerjoin(accelerometer_mins, maxhours[!, 1:3];
    on = [:Subject_ID, :wday, :hour] .=> [:Subject_ID, :wday, :maxvm_hour1],
    makeunique = false,
    validate = (false, false));

In [8]:
# get the minute data for the second highest hour found above for each person, each day
top2hour_data = DataFrames.innerjoin(accelerometer_mins, maxhours[!, [1;2;4]];
    on = [:Subject_ID, :wday, :hour] .=> [:Subject_ID, :wday, :maxvm_hour2],
    makeunique = false,
    validate = (false, false));

In [9]:
# combine top 1 hour and top 2 hour data together
top2hour_data = vcat(tophour_data, top2hour_data);

In [10]:
# sum steps over each 5 minutes to use as outcome variable
summarized_5min = combine(DataFrames.groupby(top2hour_data, 
        [:Subject_ID, :wday, :hour, :mingroup]),
    :day_worn => first => :day_worn, :season => first => :season, :steps => sum => :steps);

In [11]:
# add the daily summary data
top2hour_sumdatacomb = DataFrames.leftjoin(summarized_5min, sum_data; on = [:Subject_ID, :wday, :day_worn, :season], makeunique = false,
         indicator = nothing, validate = (false, false));

In [12]:
# add the smoking data
top2hourdata = DataFrames.leftjoin(top2hour_sumdatacomb, smoking_pub; on = [:Subject_ID, :dbGaP_Subject_ID], makeunique = false,
         indicator = nothing, validate = (false, false));

In [13]:
# Don't run twice 

top2hourdata[!, :steps] = Float64.(top2hourdata[!, :steps]);
top2hourdata[!, :smoker] = map(x -> ismissing(x) ? missing : x == 1 ? "Never" : x == 2 ? "Past" : "Current",
    top2hourdata[!, :smoke])
top2hourdata[!, :smoker] = levels!(CategoricalArray(top2hourdata[!, :smoker]),
    ["Never"; "Past"; "Current"]);

top2hourdata[!, :RACE] = map(x -> ismissing.(x) ? missing : titlecase(String(x)),
    top2hourdata[!, :RACE])
top2hourdata[!, :RACE] = levels!(CategoricalArray(top2hourdata[!, :RACE]),
    ["White"; "African American"; "Asian"; "Hispanic"; "Native American"; "Other"])

top2hourdata[!, :wday] = levels!(CategoricalArray(top2hourdata[!, :wday]),
    ["Sun"; "Mon"; "Tues"; "Wed"; "Thurs"; "Fri"; "Sat"])

top2hourdata[!, :season] = map(x -> ismissing(x) ? missing : x == 1 ? "Winter" :
    x == 2 ? "Spring" : x== 3 ?  "Summer" : "Autumn", top2hourdata[!, :season]);

top2hourdata[!, :Weekend] = map(x -> ismissing(x) ? missing :
    x in ["Mon"; "Tues"; "Wed"; "Thurs"; "Fri"] ? "Weekday" :
    "Weekend", top2hourdata[!, :wday]);

# make names more presentable to final names 
renamenames = ["Subject_ID"
 "Wday"
 "Hour"
 "mingroup"
 "Day_worn"
 "Season"
 "Steps"
 "dbGaP_Subject_ID"
 "ep60_maxsteps"
 "m_total"
 "sum_valid"
 "EP60_total_steps"
 "Compliant"
 "Race"
 "Stairs"
 "Genhealth"
 "BMI"
 "Age"
 "Smoke"
 "Smoker"
 "Weekend"] 
rename!(top2hourdata, renamenames);

In [14]:
# log10 transform (steps + 0.5)
# keepinds = findall(top2hourdata[!, :Steps] .> 0.0)
top2hour_restricted = top2hourdata #[keepinds, :]
top2hour_restricted[!, :Transformed_steps] = log10.(0.5 .+ top2hour_restricted[!, :Steps]);
# optionally save this dataset 
# CSV.write("WHS_final_cleaned.csv", top2hour_restricted);

## Analysis

As stated in the paper, we use the Knitro solver. If you do not have access to the knitro solver, you can remove solver and KNITRO and it will run, with slightly different but very similar results.

The following produce the results found in Table 3 of the paper. 

In [15]:
using DataFrames, CSV, WiSER, MixedModels, KNITRO
ENV["COLUMNS"]=1000 #extends the number of columns printed when displaying a dataframe. 

#load in data
# WHSdata = DataFrame!(CSV.File("WHS_final_cleaned.csv"));
WHSdata = top2hour_restricted

# set reference levels
WHSdata[!, :Smoker] = levels!(CategoricalArray(WHSdata[!, :Smoker]),
    ["Never"; "Past"; "Current"]);

WHSdata[!, :Race] = levels!(CategoricalArray(WHSdata[!, :Race]),
    ["White"; "African American"; "Asian"; "Hispanic"; "Native American"; "Other"])

WHSdata[!, :Wday] = levels!(CategoricalArray(WHSdata[!, :Wday]),
    ["Sun"; "Mon"; "Tues"; "Wed"; "Thurs"; "Fri"; "Sat"]);

In [16]:
# Write a function to compare mixed models with WiSER
function comparemixedmodel(mixedmodel, wsvarmodel)
    coefnames = MixedModels.coefnames(mixedmodel)
    mixedbeta = mixedmodel.β
    mixedbetapval = MixedModels.coeftable(mixedmodel).cols[4]
    wsvarbeta = wsvarmodel.β
    wsvarbetapval = WiSER.coeftable(wsvarmodel).cols[4][1:wsvarmodel.p] 
    return DataFrame(coefnames = coefnames, mixedbeta = mixedbeta,
        mixedbetapval = mixedbetapval, wsvarbeta = wsvarbeta,
        wsvarbetapval = wsvarbetapval)
end

comparemixedmodel (generic function with 1 method)

The following constructs and fits the model. The `Optimization unsuccessful` warnings can be ignored because KNITRO by default uses a very stringent convergence criterion. FeasibleApproximate indicates the solution is adequate. Other nonlinear optimization solvers such as IPOPT will return `Optimal` status.

In [19]:
wisermodel_transformed = WSVarLmmModel(
    @formula(Transformed_steps ~ 1 + BMI + Wday + Hour + 
                Race + Stairs + Age + Smoker + Season + m_total),
    @formula(Transformed_steps ~ 1 + Day_worn), 
    @formula(Transformed_steps ~ 1 + BMI + Wday + Hour + 
                Race + Age + Smoker + m_total), 
                :Subject_ID, WHSdata);
@time WiSER.fit!(wisermodel_transformed, KNITRO.KnitroSolver(outlev=0, ftol = 1e-7), parallel = false, runs = 4)

### IF NO KNITRO LICENSE, comment out last line above and run:
# solver = Ipopt.IpoptSolver(print_level=0, watchdog_shortened_iter_trigger=3, max_iter=100)
# @time WiSER.fit!(wisermodel_transformed, solver, parallel = false, runs = 4)

run = 1, ‖Δβ‖ = 0.051856, ‖Δτ‖ = 0.334485, ‖ΔL‖ = 0.004859, status = Optimal, time(s) = 7.545928
run = 2, ‖Δβ‖ = 0.046496, ‖Δτ‖ = 0.041847, ‖ΔL‖ = 0.002825, status = FeasibleApproximate, time(s) = 7.390264


┌ Warning: Optimization unsuccesful; got FeasibleApproximate; run = 2
└ @ WiSER /home/cgerman/.julia/packages/WiSER/tXr2S/src/fit.jl:63


run = 3, ‖Δβ‖ = 0.001574, ‖Δτ‖ = 0.007442, ‖ΔL‖ = 0.000047, status = FeasibleApproximate, time(s) = 7.376520


┌ Warning: Optimization unsuccesful; got FeasibleApproximate; run = 3
└ @ WiSER /home/cgerman/.julia/packages/WiSER/tXr2S/src/fit.jl:63


run = 4, ‖Δβ‖ = 0.000475, ‖Δτ‖ = 0.001376, ‖ΔL‖ = 0.000026, status = FeasibleApproximate, time(s) = 7.617126


┌ Warning: Optimization unsuccesful; got FeasibleApproximate; run = 4
└ @ WiSER /home/cgerman/.julia/packages/WiSER/tXr2S/src/fit.jl:63


 32.247652 seconds (13.61 k allocations: 833.172 KiB)



Within-subject variance estimation by robust regression (WiSER)
Number of individuals/clusters: 15390
Total observations: 2534015

Fixed-effects parameters:
────────────────────────────────────────────────────────────────────────
                                 Estimate   Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────────────────────
β1: (Intercept)               2.5266       0.0352249     71.73    <1e-99
β2: BMI                      -0.0169322    0.000413179  -40.98    <1e-99
β3: Wday: Mon                 0.0844072    0.00342547    24.64    <1e-99
β4: Wday: Tues                0.0678324    0.00352431    19.25    <1e-81
β5: Wday: Wed                 0.0653401    0.00361662    18.07    <1e-72
β6: Wday: Thurs               0.0573685    0.00361629    15.86    <1e-55
β7: Wday: Fri                 0.0724162    0.00358578    20.20    <1e-89
β8: Wday: Sat                 0.0729713    0.00344209    21.20    <1e-99
β9: Hour                     -0.0036998

In [18]:
@time mixedmodel_transformed = fit(LinearMixedModel, 
        @formula(Transformed_steps ~ 1 + BMI + Wday + Hour + 
            Race + Stairs + Age + Smoker + Season + m_total + (1 + Day_worn|Subject_ID)),
        WHSdata)

 24.091412 seconds (55.56 M allocations: 9.680 GiB, 24.77% gc time)


Linear mixed model fit by maximum likelihood
 Transformed_steps ~ 1 + BMI + Wday + Hour + Race + Stairs + Age + Smoker + Season + m_total + (1 + Day_worn | Subject_ID)
     logLik        -2 logLik          AIC             BIC       
 -2.80056306×10⁶  5.60112612×10⁶  5.60117812×10⁶   5.6015095×10⁶

Variance components:
              Column     Variance     Std.Dev.    Corr.
Subject_ID (Intercept)  0.0977251933 0.312610290
           Day_worn     0.0026893161 0.051858616 -0.66
Residual                0.5199583695 0.721081389
 Number of obs: 2534015; levels of grouping factors: 15390

  Fixed-effects parameters:
──────────────────────────────────────────────────────────────────────
                            Estimate    Std.Error     z value  P(>|z|)
──────────────────────────────────────────────────────────────────────
(Intercept)              2.47888      0.0307041     80.7346     <1e-99
BMI                     -0.0169206    0.000402285  -42.0612     <1e-99
Wday: Mon                0.0

In [20]:
dfcompare_transformed = comparemixedmodel(mixedmodel_transformed, wisermodel_transformed) 

,coefnames,mixedbeta,mixedbetapval,wsvarbeta,wsvarbetapval
,String,Float64,Float64,Float64,Float64
1,(Intercept),2.47888,0.0,2.5266,0.0
2,BMI,-0.0169206,0.0,-0.0169322,0.0
3,Wday: Mon,0.0841649,0.0,0.0844072,4.58862e-134
4,Wday: Tues,0.0675625,4.32151e-288,0.0678324,1.49543e-82
5,Wday: Wed,0.06496,7.49177e-259,0.0653401,5.83866e-73
6,Wday: Thurs,0.0570493,1.57524e-200,0.0573685,1.1264e-56
7,Wday: Fri,0.0722413,0.0,0.0724162,1.07502e-90
8,Wday: Sat,0.0735222,0.0,0.0729713,9.61188e-100
9,Hour,-0.0029124,1.76941e-112,-0.00369984,1.08614e-46


#### Supplementary Table S.3

The following obtains the results of summary statistics found in Supplementary Table S.3.

In [21]:
using DataFrames, CSV, StatsBase, Statistics

keepvars = [:BMI; :Steps; :Wday; :Hour; :Race; :Age; :Smoker; :Season; :Day_worn; :m_total; :Stairs]
# WHSdata = DataFrame!(CSV.File("WHS_final_cleaned.csv"));
descrstats = dropmissing(WHSdata, keepvars)
describe(descrstats[!, [:Steps; :m_total; :BMI; :Genhealth; :Age; :Day_worn; :Season;
                :Stairs]], :mean, :std, :min, :q25, :median, :q75, :max)

,variable,mean,std,min,q25,median,q75,max
,Symbol,Union…,Union…,Any,Union…,Union…,Union…,Any
1,Steps,85.5004,131.629,0.0,13.0,42.0,89.0,1173.0
2,m_total,879.132,134.594,1,826.0,895.0,953.0,1440
3,BMI,26.1614,4.97878,13.7318,22.5964,25.3402,28.8983,58.4195
4,Genhealth,2.0305,0.757723,1,2.0,2.0,3.0,4
5,Age,71.4786,5.65208,62,67.0,70.0,75.0,89
6,Day_worn,3.95728,1.98653,1,2.0,4.0,6.0,8
7,Season,,,Autumn,,,,Winter
8,Stairs,2.60383,1.48124,1,1.0,2.0,4.0,6


In [22]:
# Race
countmap(combine(DataFrames.groupby(descrstats, :Subject_ID), :Race => first)[!, 2]),
proportionmap(combine(DataFrames.groupby(descrstats, :Subject_ID), :Race => first)[!, 2])

(Dict{CategoricalValue{String,UInt32},Int64}("African American" => 228,"Asian" => 177,"Other" => 17,"Native American" => 27,"White" => 14806,"Hispanic" => 135), Dict{CategoricalValue{String,UInt32},Float64}("African American" => 0.014814814814814815,"Asian" => 0.011500974658869395,"Other" => 0.0011046133853151398,"Native American" => 0.0017543859649122807,"White" => 0.962053281351527,"Hispanic" => 0.008771929824561403))

In [23]:
# Days worn
mean(combine(DataFrames.groupby(descrstats, :Subject_ID), :Day_worn => maximum)[!, 2]), 
std(combine(DataFrames.groupby(descrstats, :Subject_ID), :Day_worn => maximum)[!, 2])

(6.867511371020143, 0.5465441309440071)

In [24]:
# Day of week
countmap(descrstats[!, :Wday]), proportionmap(descrstats[!, :Wday])

(Dict{CategoricalValue{String,UInt32},Int64}("Thurs" => 362912,"Wed" => 363936,"Tues" => 360287,"Sun" => 360881,"Mon" => 361726,"Fri" => 362798,"Sat" => 361475), Dict{CategoricalValue{String,UInt32},Float64}("Thurs" => 0.14321620037766153,"Wed" => 0.14362030216869276,"Tues" => 0.1421802949074887,"Sun" => 0.14241470551673924,"Mon" => 0.14274816842047108,"Fri" => 0.1431712124829569,"Sat" => 0.1426491161259898))

In [25]:
# Season
countmap(descrstats[!, :Season]), proportionmap(descrstats[!, :Season])

(Dict("Summer" => 817730,"Autumn" => 611001,"Spring" => 597968,"Winter" => 507316), Dict("Summer" => 0.3227013257616865,"Autumn" => 0.2411197250213594,"Spring" => 0.2359765036907832,"Winter" => 0.20020244552617092))

In [26]:
# Smoking Status
countmap(combine(DataFrames.groupby(descrstats, :Subject_ID), :Smoker => first)[!, 2]),
    proportionmap(combine(DataFrames.groupby(descrstats, :Subject_ID), :Smoker => first)[!, 2])

(Dict{CategoricalValue{String,UInt32},Int64}("Past" => 7082,"Current" => 548,"Never" => 7760), Dict{CategoricalValue{String,UInt32},Float64}("Past" => 0.46016894087069526,"Current" => 0.035607537361923326,"Never" => 0.5042235217673814))